<a href="https://colab.research.google.com/github/YangchenHuang/Business_Analytics/blob/master/Assignments/Yangchen_Huang_HW2_(IEOR_E4650_Spring_2020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IEOR 4650  Business Analytics (Spring 2020)**

**Homework 2 [Total points: 60]**

**Due: 11:59 PM, Feb 26**

Homework 2 consists of Python programming practices and the applications to 
* Linear regression. 
* Model selection 

The steps to finish the homework:

*   Step 1: Answer your homework on Google Colab.
*   Step 2: Submit the copy on Google Assignment

Before you submit your assignment, make sure to re-run your code from the beginning. (You can do so by first clicking Runtime/Reset All Runtimes and then clicking Runtime/Run all)

In [0]:
#This imports all the packages you need for the homework
#Please run this first


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#import modules

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler


##Setting
In this exercise, let's continue working with a dataset that explores the store sales.

Before doing the assignment, please study this dataset [here](https://www.kaggle.com/c/rossmann-store-sales/data)


For this study, we are interested in finding a good model that gives good prediction performance. 

* Especially, we use RMSPE as our accuracy measurement. RMSPE is defined as following:
$$RMSPE = \sqrt{\frac{1}{{N}}{(\frac{y_i-\widehat{y_i}}{y_i})^2}}$$
* In addition, any day and store with 0 actual sales is ignored in scoring.

In [2]:
#Load the dataset
link="https://drive.google.com/open?id=1Q6J0q4tlWJ7TajX6hMSvZ8tK3CyDD8QY"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Sales.csv')  
SALE = pd.read_csv('Sales.csv')
SALE.head(5)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


*You might see a warning when importing the data. This is because StateHoliday has both values in string and numbers. No worries this for now.*

In [3]:
link="https://drive.google.com/open?id=1g6URzUJnhCLOtg0a9TTHC18KYcqwxvoc"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile_store.csv')  
STORE = pd.read_csv('myfile_store.csv')
STORE.head(5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Q1 [7 points]

1. Merge two tables based on "Store" column

2. Select only the first 100 stores for this study

3. This study only cares about the accuracy performed on the data with Sales higher than 0. **Delete all the sales records with sales=0**.

4. Print out the unique values you have in "StateHoliday" column. If you see anything that goes wrong, fix the issue. After that, print out the unique values again in "StateHoliday" column. [You might find this [link](https://www.geeksforgeeks.org/python-pandas-series-str-replace-to-replace-text-in-a-series/) useful]


In [5]:
# Merge two tables based on "Store" column
Study=pd.merge(SALE, STORE, on="Store")
Study

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"


In [14]:
# Select only the first 100 stores for this study
Study=Study[Study["Store"]<=100]
Study

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91251,100,6,2013-01-05,7973,681,1,0,0,0,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN
91252,100,5,2013-01-04,7129,620,1,0,0,1,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN
91253,100,4,2013-01-03,7398,672,1,0,0,1,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN
91254,100,3,2013-01-02,6969,687,1,0,0,1,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN


In [15]:
# Delete all the sales records with sales=0.
Study=Study[Study["Sales"]!=0]
Study

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91249,100,1,2013-01-07,10609,823,1,1,0,0,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN
91251,100,6,2013-01-05,7973,681,1,0,0,0,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN
91252,100,5,2013-01-04,7129,620,1,0,0,1,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN
91253,100,4,2013-01-03,7398,672,1,0,0,1,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN


In [18]:
# Print out the unique values you have in "StateHoliday" column. If you see anything that goes wrong, fix the issue. After that, print out the unique values again in "StateHoliday" column.
Study["StateHoliday"].unique()

array(['0', 0, 'a', 'b', 'c'], dtype=object)

In [0]:
Study=Study["StateHoliday"].replace(0, "0")
Study["StateHoliday"].unique()

## Q2 [3 points]


Split our data into three segments. Instead of a random split, we will split our data based on the time window:

(1) Training: Time window 2013-01-01 to 2015-03-31  

(2) Validation: Time window 2015-04-01 to 2015-05-31 

(3) Testing: Time window 2015-06-01 to 2015-07-31

Print out how many records you have for each set.

In [0]:
#You might find the following sample code helpful 
#X=X.astype({"Date":"datetime64"})
#X["Date"]<"2015-01-02"


##Q3 [8 points]

Now, it is time to practice some model selection techniques. Let's suppose that we want to consider the following model as the most complicated functional form.

$log(Sales) \sim Promo+Promo2+SchoolHoliday+Promo:SchoolHoliday+Promo2:SchoolHoliday$

Here, ":" indicates an interaction term.

Based on cross-validation, choose the best model using **backward stepwise model selection starting from the full model listed above**. 
* Report RMSPE of each model and the variable to be dropped in each round of model selection
[You can do this manually]

* After selecting the final model, report the RMSPE on the test set.

##Q4 [10 points]


For this question, let's consider a more complicated model  

$np.log(Sales)\sim C(DayOfWeek)+C(SchoolHoliday)+C(StateHoliday)+C(MonthOfYear)+C(Promo)+C(Promo2)+Days$

Here, 
* C() indicating we will treat the variable as a categorical/dummy variable. 
* Days indicates the number of days since the first observation in the dataset (2013-01-01). For example, if the observation date is 2013-01-02, Days should be 1.

Once we specify "Date" as the datetime type. We can easily extract the month information. See the link [here](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetimelike-properties)

* On the training set, estimate this model using OLS

* Report how many independent variables are statistically significant

* Report the RMSPE on the validation set. 





In [0]:
#you might find the following command useful
#basedate = pd.Timestamp('2013-01-01')
#You can then use  Date series - basedate to get the difference in days


##Q5 [14 points]

* Add L1 penalty term to the regression in Q4. Plot the relationship between $\alpha$ and the number of independent variables kept (the number of independent variables with $\hat{\beta}$ not equal to 0). 

* Plot the relationship between $\alpha$ and RMSPE on the Validation set. 

* Report the best $\alpha$ value and the corresponding RMSPE on the validation set.

* Report which variables are kept for the best model

[Remember to demean the y-value. Also, remember to standardize the x variables using StandardScaler]





##Q6 [10 points]

* Add L2 penalty term to the regression in Q4. 

* Plot the relationship between $\alpha$ and RMSPE on the validation set. 

* Report the best $\alpha$ value and the corresponding RMSPE on the validation set.

* Report which variables are kept for the best model

* You might find that the $\alpha$ values are on very different scales in Q5 and Q6. Find out why this is the case by doing a little research yourself. [The answer should be pretty straightforward]
 
[Remember to demean the y-value. Also, remember to standardize the x variables using StandardScaler]




##Q7 [8 points]

* Use ElasticNet with $L1=0.25, 0.5, 0.75$. Discuss if you can further improve the model performance.

* Report the RMSPE on the testing set based on the best model chosen.

[Remember to demean the y-value. Also, remember to standardize the x variables using StandardScaler]



